## [计算机网络发展简介](https://github.com/jackfrued/Python-100-Days/blob/master/Day01-15/14.网络编程入门和网络应用开发.md)
* OSI七层模型
* TCP/IP四层模型
* NAT服务
* TCP协议(1.握手,校验与重传机制 2.流量控制,用于匹配传输速度 3.拥塞控制)
* 网络应用模式(1. C/S,B/S 2.去服务器中心化的网络应用模式,P2P?)
* HTTP协议 [《HTTP 协议入门》](http://www.ruanyifeng.com/blog/2016/08/http.html)
* Json


```XML
<?xml version="1.0" encoding="UTF-8"?>
<message>
	<from>Alice</from>
	<to>Bob</to>
	<content>Will you marry me?</content>
</message>
```
```Json
{
    "from": "Alice",
    "to": "Bob",
    "content": "Will you marry me?"
}
```

### 基于HTTP的requests库

In [13]:
# author:TYUT-Lmy
# date:2021/12/14
# description:
from threading import Thread
import requests
import os


class DownloadHandler(Thread):
    def __init__(self, url, path):
        super().__init__()
        self.url = url
        self.path = path

    def run(self):
        filename = self.url[self.url.rfind("/") + 1:]  # 将url最后一个/后面所有的文件设置为文件名
        response = requests.get(self.url)

        with open(self.path + "/" + filename, 'wb') as f:
            f.write(response.content)


def check_path(path):
    if not os.path.exists(path):
        os.mkdir(path)


def main():
    # 通过requests模块的get函数获取网络资源
    # 下面的代码中使用了天行数据接口提供的网络API
    # 要使用该数据接口需要在天行数据的网站上注册
    # 然后用自己的Key替换掉下面代码的中APIKey即可
    resp = requests.get('https://api.tianapi.com/esports/index?key=acf20985280976b251ac3dbbc91&num=10')
    data_model = resp.json()  # 返回json数据,需要解析为字典
    path = "./res"
    check_path(path)
    for mm_dict in data_model["newslist"]:
        url = mm_dict['picUrl']
        DownloadHandler("http:" + url, path).start()
    print(f"下载{len(data_model['newslist'])}张图片完成")


if __name__ == '__main__':
    main()


存在
下载10张图片完成


## 基于传输层协议的套接字编程
实际开发中使用的套接字可以分为三类：流套接字（TCP套接字）、数据报套接字和原始套接字。

### TCP套接字
顾名思义,使用TCP协议提供的传输服务实现网络通信,Python中创建socket对象,指定type属性是SOCK_STREAM使用TCP套接字,创建后需要将对象绑定到指定的IP和端口上.
端口的取值范围是0~65535，而1024以下的端口我们通常称之为“著名端口”（留给像FTP、HTTP、SMTP等“著名服务”使用的端口，有的地方也称之为“周知端口”），自定义的服务通常不使用这些端口，除非自定义的是HTTP或FTP这样的著名服务。
* family=AF_INET - IPv4地址
* family=AF_INET6 - IPv6地址
* type=SOCK_STREAM - TCP套接字
* type=SOCK_DGRAM - UDP套接字
* type=SOCK_RAW - 原始套接字

#### 下面的代码实现了一个提供时间日期的服务端开启。
单线程

In [20]:
# author:TYUT-Lmy
# date:2021/12/14
# description:
from socket import SOCK_STREAM, AF_INET
from datetime import datetime


def main():
    # 1.创建套接字并指定使用哪种服务
    server = socket(family=AF_INET, type=SOCK_STREAM)
    # 2.绑定IP地址和端口(打开该端口,作为服务端)
    ip_port = ('127.0.0.1', 6789)
    server.bind(ip_port)
    # 3.开启监听,参数可以理解为连接队列的大小
    server.listen(2)
    print("服务器开启了监听")
    while True:
        # 4.通过循环接收客户端的连接,并作出相应的处理
        # accept 方法是一个阻塞方法, 如果没有客户端连接到服务器端口,则不会向下执行,返回一个元组,其中第一个元素是客户端对象,第二个元素是客户端的IP和端口
        client, addr = server.accept()
        print(f"{str(addr)}连接了服务器.")

        # 5.检测到有客户端连接,则发送数据
        client.send(f"当前北京时间{str(datetime.now())}\n".encode('utf-8'))

        print(f"向服务器发送了日期信息.")

        # 6.发送完成,断开客户端
        client.close()


if __name__ == "__main__":
    main()

OSError: [Errno 48] Address already in use

下面代码实现TCP客户端的服务

In [21]:
from socket import socket

def main():
    # 1.创建套接字对象默认使用IPv4和TCP协议
    client = socket()
    # 2.连接到服务器(需要指定IP地址和端口)
    client.connect(('192.168.1.2', 6789))
    # 3.从服务器接收数据
    print(client.recv(1024).decode('utf-8'))
    client.close()


if __name__ == '__main__':
    main()

ConnectionRefusedError: [Errno 61] Connection refused

In [23]:
from time import sleep
from base64 import b64encode
from json import dumps
from threading import Thread


def get_data() -> str:
    """
    guido.jpg是一个图片,即二进制数据,不能够通过json传输,需要将二进制编码
    :return: 图片的二进制数据
    """
    DATA = ''
    with open("./res/guido.jpg", 'rb') as f:
        DATA = b64encode(f.read()).decode('utf-8')
    return DATA


class FrileTransferHandler(Thread):
    def __init__(self, client, addr, data):
        super().__init__()
        self.client = client
        self.addr = addr
        self.data = data

    def run(self):
        # 生成包含文件名和内容的字典
        my_dict = {'file_name': 'guido.jpg', 'file_data': self.data}
        # 将字典通过dumps转为json 用于传输
        json_str = dumps(my_dict)
        self.client.send(json_str)
        sleep(10)
        self.client.close()
        print(f"{self.addr}已经关闭")


def main():
    data = get_data()  # 获取图片的数据

    # 配置服务器
    server = socket()
    port = 7879
    ip_port = ("127.0.0.1", port)
    server.bind(ip_port)
    server.listen(10)
    print(f'服务器启动开始监听{port}端口')

    # 设置监听动作
    while True:
        sleep(.1)
        client, addr = server.accept()
        #不断启动线程来对客户端的请求进行处理
        FrileTransferHandler(client, addr, data)

if __name__ =='__main__':
    main()


服务器启动开始监听7879端口


KeyboardInterrupt: 

In [ ]:
from socket import socket
from json import loads
from base64 import b64decode
import os

def check_dir(dir):
    if not os.path.exists(dir):
        os.mkdir(dir)

def main():
    client = socket()
    port = 7879
    ip_port = ("127.0.0.1", port)
    client.connect(ip_port)
    # 定义一个保存二进制数据的对象
    in_data = bytes()
    # 由于不知道服务器发送的数据有多大每次接收1024字节
    data = client.recv(1024)
    while data:
        # 将收到的数据拼接起来
        in_data += data
        data = client.recv(1024)
    # 将收到的二进制数据解码成JSON字符串并转换成字典
    # loads函将JSON字符串转成字典对象
    my_dict = loads(in_data.decode('utf-8'))
    filename = my_dict['filename']
    filedata = my_dict['filedata'].encode('utf-8')

    dir = "./res/new"
    check_dir(dir)
    with open(dir + filename, 'wb') as f:
        # 将base64格式的数据解码成二进制数据并写入文件
        f.write(b64decode(filedata))
    print('图片已保存.')

if __name__ == '__main__':
    main()

### UDP套接字
TCP和UDP都是提供端到端传输服务的协议
TCP可靠,但是开销大
UDP不进行握手,直接发送数据,并且不对接收到的信息进行完整性判断
适用场景:发送语音,视频等开销较大,且能容忍稍微的花屏卡顿的活动

#### 案例:电子邮件
使用smtplib模块,建立在TCP协议的基础上进行邮件发送

In [ ]:
from smtplib import SMTP
from email.header import Header
from email.message import Message
from email.mime.text import MIMEText

def main():
    sender = "betterlmy@icloud.com"
    receiver = "comeonlmy@gmail.com"
    message = MIMEText(_text="Python发送的邮件",_subtype='plain',_charset='utf-8')
    message['From'] = Header('Zane')
    message['To'] = Header('李梦洋')
    message['Subject']=Header("Python测试",'utf-8')
    smtper = SMTP('smtp.126.com')
    smtper.login(sender, 'secretpass')
    smtper.sendmail(sender, receiver, message.as_string())
    print('邮件发送完成!')


if __name__ == '__main__':
    main()